# Load Raw Data And Compute MLDS

In [ ]:
import numpy as np
import itertools
import pandas as pd
from jax import grad, jit, vmap
import jax.numpy as jnp
import jax.scipy as jscipy
import matplotlib.pyplot as plt

In [ ]:
# Load Experiments Raw Data from github
url = 'https://raw.githubusercontent.com/paudauo/BBDD_Affine_Transformations/refs/heads/main/answers.csv'
BBDD = pd.read_csv(url,header=0)

In [ ]:
# Choose the distortion and the image
im_ref = 1  # Reference Image (1,...,24)
transf = ['gaussian_noise'] #Distortion  (['rotation'] ['translation'] ['scale'] ['gaussian_noise'])

Nimages = 10 # Number of distortion levels

# Load the specific experiment
BBDD = BBDD[BBDD['image_id_01'] == im_ref]
BBDD = BBDD[BBDD['transformation'].isin(transf)]

exps = BBDD[['distortion_level_11','distortion_level_12','distortion_level_21','distortion_level_22','answer']].values

exps = exps-1
exps = exps.astype('int')
exps = exps[exps[:,4]>-1]

In [ ]:
Np = 4000 # Number of optimization steps
psi = np.linspace(0,1,Nimages) # Initial parametes
sig = 0.1 # Initial parametes

In [ ]:
# Function to compute the loglikelihood
def LL(x,psi,sig):
    LL = 0

    for n in range(x.shape[0]):
        # comparisons
        D =jnp.abs(psi[x[n,0]]-psi[x[n,1]])-jnp.abs(psi[x[n,2]]-psi[x[n,3]])

        # Choice Proability
        pp = jscipy.stats.norm.cdf(D/sig)

        # Choose 0 (p) or 1 (1-p)
        aa = (x[n,4]-pp)
        bb = ((-1)**(1+x[n,4]))

        # LogLikelihood
        LL = LL + jnp.log(aa*bb)

    return LL

In [ ]:
# Gradient descent learning of the MLSD pareameters

D_loss_D_psi = grad(LL,1)
D_loss_D_sig = grad(LL,2)

D_loss_D_psi = jit(D_loss_D_psi)
D_loss_D_sig = jit(D_loss_D_sig)

lr = 0.00001

BS = 20
cada = 100

LLs = np.zeros((Np,))
for ii in range(Np):

        iii = np.random.permutation(exps.shape[0])[0:BS]
        DD = D_loss_D_psi(exps[iii,:],psi,sig)
        DDs = D_loss_D_sig(exps[iii,:],psi,sig)

        psi[1:-1] =  psi[1:-1] + lr*DD[1:-1]
        sig = sig + lr*DDs

        LLs[ii] = LL(exps,psi,sig)
        if np.mod(ii,cada)==0:
            print(ii,np.round(LLs[ii],5),np.round(psi,3),np.round(sig,3))

        if np.isnan(LLs[ii]):
            break

0 -327.91336 [0.    0.111 0.222 0.333 0.444 0.556 0.666 0.778 0.889 1.   ] 0.101
100 -170.13516 [0.    0.11  0.245 0.354 0.464 0.575 0.66  0.773 0.876 1.   ] 0.19600001
200 -153.99384 [0.    0.11  0.258 0.365 0.473 0.588 0.659 0.772 0.869 1.   ] 0.22600001
300 -145.38272 [0.    0.108 0.27  0.375 0.48  0.599 0.657 0.772 0.864 1.   ] 0.24700001
400 -140.63817 [0.    0.107 0.279 0.382 0.486 0.608 0.655 0.774 0.861 1.   ] 0.261
500 -136.97394 [0.    0.107 0.286 0.389 0.492 0.617 0.654 0.776 0.858 1.   ] 0.272
600 -134.46623 [0.    0.106 0.293 0.396 0.497 0.625 0.653 0.777 0.856 1.   ] 0.27800003
700 -132.26297 [0.    0.105 0.3   0.402 0.503 0.634 0.654 0.778 0.854 1.   ] 0.284
800 -130.35901 [0.    0.104 0.305 0.407 0.509 0.641 0.654 0.78  0.851 1.   ] 0.29000002
900 -128.76437 [0.    0.104 0.312 0.411 0.513 0.649 0.653 0.78  0.851 1.   ] 0.29500002
1000 -127.42954 [0.    0.104 0.316 0.417 0.518 0.655 0.653 0.781 0.85  1.   ] 0.29900002
1100 -126.23181 [0.    0.103 0.322 0.423 0.521 0.659 

In [ ]:

# Plot the computed MLSD response
plt.figure()
SIGS = sig*np.ones(*psi.shape)
xxx = np.linspace(0,psi.shape[0]-1,psi.shape[0])
plt.errorbar(xxx,psi,SIGS)

plt.xlabel('Distortion level')
plt.ylabel('Response')